<a href="https://colab.research.google.com/github/lrssv/ComputacaoEvolutiva/blob/master/Fitness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model for classifying the victim's gender using an evolutionary algorithm to select features.

* ## Fitness

In [ ]:
!pip install category_encoders

## Imports

In [9]:
import pandas as pd
import numpy as np
import random
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pandas import read_csv 
from category_encoders import TargetEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt

## Scripts

In [10]:
def database(url):
  df_encoded = pd.read_csv(url)
  df_encoded = df_encoded.drop(columns=['Unnamed: 0'])

  return df_encoded

In [17]:
def fitness(population,df,models):
  result = []

  x = df[df.columns.difference(['Victim Sex'])]
  y = df[['Victim Sex']]
  
  x_train, x_test, y_train, y_test = np.array(train_test_split(x, y, test_size=0.2, stratify=y, random_state=42))

  x_train = np.array(x_train)
  x_test = np.array(x_test)
  y_train = np.ravel(y_train)
  y_test = np.array(y_test)

  for p in population:
    model = models[p[1]]
    ind = [i==1 for i in p[1:]] 
    model.fit(x_train[:,ind],y_train)
    y_pred = model.predict(x_test[:,ind])
    accuracy = accuracy_score(y_test,y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    fit_value =  rmse + np.sum(p[1:])

    result.append([p,fit_value])

  return result

## Setups

In [26]:
with open('/content/drive/My Drive/Data Files/initial_population.txt', "rb") as fp:   
  population = pickle.load(fp)

In [14]:
models = [DecisionTreeClassifier(max_depth=10),DecisionTreeClassifier(max_depth=3),DecisionTreeClassifier(max_depth=14),RandomForestClassifier(max_depth=10)]

In [15]:
df_encoded = database('/content/drive/My Drive/Data Files/df_model_victim_encoded')

In [18]:
model_fitness = fitness(population,df_encoded,models)

In [29]:
#export 
with open("/content/drive/My Drive/Data Files/fitness.txt", "wb") as fit: 
  pickle.dump(model_fitness, fit)